# МОДУЛЬ 3: Advanced ООП

## 1. Дополнительные dunder-методы

### 1.1. `__new__` и `super().__new__(cls, ...)`
- `__new__` вызывается до `__init__` и отвечает за **создание нового экземпляра**.
- Принимает класс (`cls`) и должен **вернуть** новый объект.
- `super().__new__(cls, ...)` вызывает `__new__` родительского класса.
- Если `__new__` вернет объект другого класса или `None`, `__init__` **не** вызывается.

Ниже простой пример, где в `__new__` мы лишь выводим сообщение и возвращаем объект, а уже в `__init__` — инициализируем атрибуты:

In [1]:
class MyClass:
    def __new__(cls, *args, **kwargs):
        print("__new__ called")
        instance = super().__new__(cls)
        return instance

    def __init__(self):
        print("__init__ called")
        self.value = 42

obj = MyClass()
print("Value:", obj.value)


__new__ called
__init__ called
Value: 42


Дополнительный пример: когда `__new__` возвращает `None`, конструктор `__init__` вызываться не будет:

In [2]:
class NoInstance:
    def __new__(cls, *args, **kwargs):
        print("No instance created")
        return None  # Возвращаем None

    def __init__(self):
        print("__init__ called")

obj = NoInstance()
print(obj)  # None

No instance created
None


### 1.2. `__del__` (финализация объекта)
- Вызывается при уничтожении объекта сборщиком мусора.
- Обычно используется для освобождения **внешних ресурсов** (файлы, сетевые соединения).
- На практике советуют использовать **контекстные менеджеры** или прочие механизмы, чтобы явным образом управлять ресурсами.

In [3]:
class Resource:
    def __init__(self, name):
        self.name = name
        print(f"Resource {self.name} acquired")

    def __del__(self):
        print(f"Resource {self.name} released")

r = Resource("DB Connection")
# Уничтожение может произойти при выходе из области видимости или при завершении программы.


Resource DB Connection acquired


## 2. Singleton, Generic

### 2.1. Шаблон Singleton
- Гарантирует, что у класса есть только **один** экземпляр.
- Информация о Singleton не была в исходных источниках, но вот пример реализации.

In [4]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

s1 = Singleton()
s2 = Singleton()
print(s1 is s2)  # True

True


### 2.2. Generic через `__class_getitem__`
- Позволяет создавать **параметризованные классы** (Generic).
- Вызывается при использовании синтаксиса `MyClass[T]`.
- Можно сохранить информацию о типе и использовать её внутри класса.

Ниже упрощенный пример, где `__class_getitem__` печатает переданный тип и динамически создает подкласс:

In [5]:
class Box:
    def __init__(self, item):
        self.item = item

    def __repr__(self):
        return f"Box({self.item})"

    def __class_getitem__(cls, item_type):
        # Можно добавить проверки, логику и т.д.
        print(f"__class_getitem__ called with: {item_type}")
        return type(f"{cls.__name__}_{item_type}", (cls,), {"_type": item_type})

# Пример использования:
IntBox = Box[int]
ib = IntBox(123)
print(ib)
print("Type stored:", ib._type)

__class_getitem__ called with: <class 'int'>
Box(123)
Type stored: <class 'int'>


## 3. Индексирование и подскрипты

### 3.1. `__class_getitem__` vs `__getitem__`, `__setitem__`, `__delitem__`
- `__getitem__`, `__setitem__`, `__delitem__` работают с **экземпляром** класса (`obj[key]`).
- `__class_getitem__` работает при индексировании самого **класса** (`ClassName[key]`).

### 3.2. Исключение `IndexError`
- Генерируется при выходе за пределы индекса.
- Цикл `for` вызывает `__getitem__`, увеличивая индекс, пока не произойдет `IndexError`.

In [6]:
class MyList:
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        if index < 0 or index >= len(self.data):
            raise IndexError("Index out of range")
        return self.data[index]

my_list = MyList([10, 20, 30])
for x in my_list:
    print(x)


10
20
30


## 4. Итерация

### 4.1. `__getitem__`, `__iter__`, `__next__`
- `__iter__` должен возвращать объект-итератор.
- `__next__` возвращает следующий элемент или возбуждает `StopIteration`.
- Если `__iter__` не определен, Python может использовать `__getitem__` для итерации (но это менее гибкий вариант).

In [7]:
class RangeIter:
    def __init__(self, start, end):
        self.current = start
        self.end = end

    def __iter__(self):
        return self  # объект сам является итератором

    def __next__(self):
        if self.current >= self.end:
            raise StopIteration
        val = self.current
        self.current += 1
        return val

for num in RangeIter(0, 3):
    print(num)

0
1
2


### 4.2. Исключение `StopIteration`
- Возбуждается, когда итератор заканчивается.
- Является сигналом для цикла `for` завершить итерацию.

## 5. Контекстные менеджеры

### 5.1. `__enter__` и `__exit__`
- `__enter__` вызывается при входе в блок `with`.
- `__exit__` вызывается при выходе из блока `with`, даже если возникло исключение.

In [8]:
class MyContext:
    def __enter__(self):
        print("Enter context")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exit context")
        if exc_type:
            print(f"An exception occurred: {exc_value}")
        # По умолчанию возвращаем None или False, чтобы не подавлять исключение.

with MyContext() as ctx:
    print("Inside with block")


Enter context
Inside with block
Exit context


### 5.2. Параметры `exc_type`, `exc_value`, `traceback`
- Передаются в `__exit__` и содержат информацию об исключении, если оно произошло.

### 5.3. Языковая конструкция `with`
- Упрощает работу с ресурсами, которые нужно освободить.
- Пример: `with open('file.txt') as f: ...`

## 6. Дескрипторы

### 6.1. Data Descriptor vs Non-data Descriptor
- **Data descriptor**: определяет методы `__get__` и `__set__`.
- **Non-data descriptor**: определяет только метод `__get__`.
- Data-дескрипторы имеют **приоритет** над атрибутами экземпляра.

### 6.2. Методы `__get__`, `__set__`, `__delete__` vs `__del__`, `__set_name__`
- `__get__`: чтение атрибута.
- `__set__`: запись атрибута.
- `__delete__`: удаление атрибута.
- `__del__`: вызов при уничтожении объекта (не путать с `__delete__`).
- `__set_name__`: вызывается при создании класса, чтобы дескриптор знал свое имя.

### 6.3. @property как Data Descriptor
- `@property` создает data-дескриптор.
- Позволяет определить логику чтения/записи/удаления атрибута через геттер, сеттер и делетер.

In [9]:
class MyDescriptor:
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

class Sample:
    attr = MyDescriptor()

s = Sample()
s.attr = 100
print(s.attr)  # 100

100


## 7. Доступ к атрибутам объекта

### 7.1. `__getattribute__` vs `__getattr__`, `__setattr__`, `__delattr__`
- `__getattribute__`: вызывается при **каждом** доступе к атрибуту.
- `__getattr__`: вызывается только если атрибут **не найден** обычным способом.
- `__setattr__`: вызывается при установке атрибута.
- `__delattr__`: вызывается при удалении атрибута.

### 7.2. Предотвращение бесконечной рекурсии
- Использовать `object.__setattr__(self, name, value)` или `super().__setattr__(name, value)` внутри `__setattr__`.
- Аналогично для `__delattr__`.

In [10]:
class AttrLogger:
    def __getattribute__(self, name):
        print(f"Getting {name}")
        return super().__getattribute__(name)

    def __setattr__(self, name, value):
        print(f"Setting {name} = {value}")
        super().__setattr__(name, value)

a = AttrLogger()
a.x = 10  # Setting x = 10
print(a.x) # Getting x -> 10

Setting x = 10
Getting x
10


## 8. `super(...)`

### 8.1. Прокси-объект: `__thisclass__`, `__self__`, `__self_class__`
- `super()` возвращает объект-прокси, который **делегирует** вызовы родительским классам.
- Прокси содержит:
  - `__thisclass__`: какой класс вызвал `super()`.
  - `__self__`: экземпляр, привязанный к вызову `super()`.
  - `__self_class__`: класс экземпляра, для которого вызван `super()`.

### 8.2. `mappingproxy` и `TypeError`
- `mappingproxy` — это неизменяемый словарь пространства имен класса.
- Попытка изменить `mappingproxy` напрямую приводит к `TypeError`.

## 9. MRO (Method Resolution Order)

### 9.1. «Ромбовидное» наследование / diamond problem
- Возникает, когда класс наследуется от двух родительских, которые в свою очередь имеют общего предка.
- Может приводить к неоднозначности в вызовах методов, если не учтён порядок.

### 9.2. C3-линеаризация
- Python использует **C3-линеаризацию** для определения порядка поиска методов (MRO).
- Гарантирует **логичный** и **предсказуемый** порядок разрешения.

In [11]:
class A:
    def test(self):
        print("A")

class B(A):
    def test(self):
        print("B")

class C(A):
    def test(self):
        print("C")

class D(B, C):
    pass

d = D()
d.test()
print(D.__mro__)

B
(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


## 10. Метаклассы

### 10.1. `type(name, bases, dict)`
- `type` — это метакласс по умолчанию.
- `type(name, bases, dict)` создаёт новый класс с переданными именем, базовыми классами и атрибутами.

In [12]:
NewClass = type("NewClass", (object,), {"attr": 123})
nc = NewClass()
print(nc.attr)

123


### 10.2. Создание собственных метаклассов
- Наследовать от `type` и переопределить методы (`__new__`, `__init__`).

In [13]:
class MyMeta(type):
    def __new__(mcs, name, bases, attrs):
        print(f"Creating class {name} with metaclass {mcs.__name__}")
        return super().__new__(mcs, name, bases, attrs)

class MyClass(metaclass=MyMeta):
    pass

# Создание экземпляра
obj = MyClass()

Creating class MyClass with metaclass MyMeta


### 10.3. Динамическое создание классов
- При помощи `type` можно создавать классы «на лету» в рантайме.

## 11. Миксины
- Миксины — это классы, которые предоставляют дополнительную функциональность, но сами по себе **не** предназначены для создания экземпляров.
- Содержат методы/атрибуты, которые «примешиваются» к другим классам через множественное наследование.
- В исходных источниках подробности про миксины не описаны, но это распространённая практика для расширения функционала. Ниже короткий пример:

In [14]:
class GreetMixin:
    def greet(self):
        print("Hello from mixin!")

class Person:
    def __init__(self, name):
        self.name = name

class FriendlyPerson(GreetMixin, Person):
    pass

fp = FriendlyPerson("Bob")
fp.greet()
print(fp.name)


Hello from mixin!
Bob
